# __Ocean regimes indicator__

Apply a Gaussian Mixtures Model to a dataset of time series

***
ds : initial dataset (lat, lon, week)

X : staked dataset (sampling, week, week_reduced)

ds_labels: unstacked final dataset (lat, lon, week)
***

In [1]:
import xarray as xr
import numpy as np
import pandas as pd

import Plotter_OR
from Plotter_OR import Plotter_OR 

from BIC_calculation_OR import *

from preprocessing import *

## Load dataset
***

- One year of SST data in the Mediterranean Sea:  SST_GLO_SST_L4_NRT_OBSERVATIONS_010_014 product

In [2]:
file_path = 'datasets/METOFFICE-GLO-SST-L4-NRT-OBS-SKIN-DIU-FV01.1_1607013925530.nc'

- One year of SST data in the North Atlantic:  SST_GLO_SST_L4_NRT_OBSERVATIONS_010_014 product

In [ ]:
#file_path = 'datasets/METOFFICE-GLO-SST-L4-NRT-OBS-SKIN-DIU-FV01.1_1608215256580.nc'

- One year of Ocean Color data in the Mediterranean Sea:  OCEANCOLOUR_GLO_CHL_L4_REP_OBSERVATIONS_009_082 product

In [ ]:
#file_path = 'datasets/dataset-oc-glo-bio-multi-l4-chl_interpolated_4km_daily-rep_1610026811620.nc'

Open dataset

In [3]:
ds = xr.open_dataset(file_path)
print(ds)
#ds

<xarray.Dataset>
Dimensions:                                        (lat: 65, lon: 192, time: 8760)
Coordinates:
  * time                                           (time) datetime64[ns] 2019-01-01T00:30:00 ... 2019-12-31T23:30:00
  * lat                                            (lat) float32 30.125 ... 46.125
  * lon                                            (lon) float32 -4.875 ... 42.875
Data variables:
    sea_surface_warm_layer_temperature_difference  (time, lat, lon) float32 ...
    sea_surface_cool_skin_temperature_difference   (time, lat, lon) float32 ...
    analysed_sst                                   (time, lat, lon) float32 ...
    mask                                           (time, lat, lon) float32 ...
Attributes:
    Conventions:                CF-1.4
    title:                      Global Diurnal Skin SST Analysis, L4 OSTIA, 0...
    summary:                    A merged, multi-sensor L4 skin SST product
    references:                 While J., Martin M.; 2013;  D

In [4]:
#important to identify time variable
#ds['time'] = ds.indexes['time'].to_datetimeindex()
var_name = 'analysed_sst'

__Quick plot__

In [ ]:
ds[var_name].isel(time=11).plot();

In [ ]:
bins = np.arange(int(ds[var_name].min().values), int(ds[var_name].max().values))
ds[var_name].plot.hist(bins=bins);

## Preprocessing
***

#### __1) Weekly mean for each pixel__

Weekly mean is useful for seasonal trends (lisser le signal): should it be an option?

In [5]:
X = ds.groupby("time.week").mean()
print(X)

/export/home1/agarciaj/anaconda3/lib/python3.7/site-packages/xarray/core/accessor_dt.py:45: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  field_values = getattr(values_as_series.dt, name).values
/export/home1/agarciaj/anaconda3/lib/python3.7/site-packages/xarray/core/nanops.py:142: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)


<xarray.Dataset>
Dimensions:                                        (lat: 65, lon: 192, week: 52)
Coordinates:
  * lon                                            (lon) float32 -4.875 ... 42.875
  * lat                                            (lat) float32 30.125 ... 46.125
  * week                                           (week) int64 1 2 3 ... 51 52
Data variables:
    sea_surface_warm_layer_temperature_difference  (week, lat, lon) float32 nan ... 0.0
    sea_surface_cool_skin_temperature_difference   (week, lat, lon) float32 nan ... 0.0
    analysed_sst                                   (week, lat, lon) float32 nan ... 275.3912
    mask                                           (week, lat, lon) float32 2.0 ... 1.0


Plot histogram in time: in plotter class?

In [ ]:
bins = np.arange(int(X[var_name].min().values), int(X[var_name].max().values),0.5)
#bins = np.arange(0, 0.3, 0.002)
histo_2d = [] 
for iweek in range(52):
#for iweek in range(12):
    hist_values, bin_edges = np.histogram(X[var_name].isel(week=iweek).values, bins=bins)
    #hist_values, bin_edges = np.histogram(ds[var_name].isel(month=iweek).values, bins=bins)
    histo_2d.append(hist_values)
    
fig, ax = plt.subplots(figsize=(12,10))

#plt.pcolormesh(bins, ds.month.values, histo_2d, cmap='Reds', edgecolors='black')
plt.pcolormesh(bins, X.week.values, histo_2d, cmap='Reds', edgecolors='black')
cbar = plt.colorbar()
ax.set_xlabel('Temperature (K)')
#ax.set_xlabel('Chlorophyll-a concentration (milligram m-3)')
ax.set_ylabel('Weeks')
cbar.ax.set_ylabel('Counts')

#### __2) Reduce lat lon dimensions to sampling dimension__

In [6]:
sampling_dims = list(X.dims)
sampling_dims.remove('week')
sampling_dims

['lat', 'lon']

In [7]:
X = X.stack({'sampling': sampling_dims})
X = X.rename_dims({'week': 'feature'})
X = X.rename({'week': 'feature'})
print(X)
#X

<xarray.Dataset>
Dimensions:                                        (feature: 52, sampling: 12480)
Coordinates:
  * feature                                        (feature) int64 1 2 ... 51 52
  * sampling                                       (sampling) MultiIndex
  - lat                                            (sampling) float64 30.12 ... 46.12
  - lon                                            (sampling) float64 -4.875 ... 42.88
Data variables:
    sea_surface_warm_layer_temperature_difference  (feature, sampling) float32 nan ... 0.0
    sea_surface_cool_skin_temperature_difference   (feature, sampling) float32 nan ... 0.0
    analysed_sst                                   (feature, sampling) float32 nan ... 275.3912
    mask                                           (feature, sampling) float32 2.0 ... 1.0


#### __3) Delate all NaN time series using mask__

In [8]:
X, mask = delate_NaNs(X, var_name=var_name)

In [9]:
X

<xarray.Dataset>
Dimensions:       (feature: 52, sampling: 5741)
Coordinates:
  * feature       (feature) int64 1 2 3 4 5 6 7 8 9 ... 45 46 47 48 49 50 51 52
  * sampling      (sampling) MultiIndex
  - lat           (sampling) float64 30.38 30.38 30.38 ... 46.12 46.12 46.12
  - lon           (sampling) float64 18.62 18.88 19.12 ... 38.38 38.62 42.88
Data variables:
    analysed_sst  (feature, sampling) float32 291.3304 291.29144 ... 275.3912

In [10]:
mask

<xarray.Dataset>
Dimensions:  (feature: 52, lat: 65, lon: 192)
Coordinates:
  * feature  (feature) int64 1 2 3 4 5 6 7 8 9 10 ... 44 45 46 47 48 49 50 51 52
  * lat      (lat) float64 30.12 30.38 30.62 30.88 ... 45.38 45.62 45.88 46.12
  * lon      (lon) float64 -4.875 -4.625 -4.375 -4.125 ... 42.38 42.62 42.88
Data variables:
    mask     (feature, lat, lon) bool False False False ... False False True

In [ ]:
#plot mask
mask['mask'].plot();

Create mask from data

In [ ]:
mask_d = X[var_name].notnull()
X = X.assign(variables={"mask_d":(('feature','sampling'), mask_d)})
print(X)
#X

Plot created mask

In [ ]:
mask_plot = X['mask_d'].unstack('sampling')
#.sortby(['lat','lon'])
mask_plot.isel(feature=11).plot();

Apply mask 

In [ ]:
stacked_mask = X['mask_d']
print(stacked_mask)
#stacked_mask

In [ ]:
X = X[var_name].where(stacked_mask == True, drop=True).to_dataset()
print(X)
#X

Recover the dataset (unravel)

In [ ]:
X_unstacked = X[var_name].unstack('sampling').to_dataset(name = var_name)
X_unstacked = X_unstacked.sortby(['lat','lon'])
print(np.shape(X_unstacked[var_name]))
# same lat and lon values in mask and in results
mask = stacked_mask.unstack()
X_unstacked = X_unstacked.reindex_like(mask)
print(np.shape(X_unstacked[var_name]))
print(X_unstacked) 

In [ ]:
X_unstacked[var_name].isel(feature=11).plot();

__There is any NaN in the dataset?__

In [ ]:
np.any(np.isnan(X[var_name].values))

#### __4) Interpolation__

Not necessary if using mask created from dataset

In [ ]:
X = X[var_name].interpolate_na(dim = 'feature', method="linear", fill_value="extrapolate").to_dataset(name = var_name)
print(X)
#X

__There is any NaN in the dataset?__

In [ ]:
np.any(np.isnan(X[var_name].values))

#### __5) Scaler__

__Check dimensions order__
***

In [ ]:
np.shape(X[var_name])

Transpose dataset if needed (sampling x features)

In [ ]:
X = X.transpose("sampling", "feature")
np.shape(X[var_name].values)

***

Apply sklearn __StandardScaler__: 
Standardize features by removing the mean and scaling to unit variance
The standard score of a sample x is calculated as:

    z = (x - u) / s
    
where u is the mean of the training samples or zero if `with_mean=False`, and s is the standard deviation of the training samples or one if `with_std=False`.
Centering and scaling happen independently on each feature by computing the relevant statistics on the samples in the training set. 

In [ ]:
from sklearn.preprocessing import StandardScaler
X_scale = StandardScaler().fit_transform(X[var_name])
X = X.assign(variables={var_name + "_scaled":(('sampling', 'feature'), X_scale)})
print(X)
#X

#### __5) PCA__

Apply __Principal component analysis__ (PCA):
Linear dimensionality reduction using Singular Value Decomposition of the data to project it to a lower dimensional space.
If `0 < n_components < 1` and `svd_solver == 'full'`, select the number of components such that the amount of variance that needs to be explained is greater than the percentage specified by n_components.

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components = 0.99, svd_solver = 'full')
pca

In [ ]:
pca = pca.fit(X[var_name + "_scaled"])

In [ ]:
fig, ax = plt.subplots()
pb = plt.bar(range(pca.n_components_), pca.explained_variance_ratio_)
ax.set_xlabel('n_components')
ax.set_ylabel('Percentage')
ax.set_title('Percentage of variance explained by each of the selected components')

In [ ]:
X_reduced = pca.transform(X[var_name + "_scaled"])
np.shape(X_reduced)

In [ ]:
X = X.assign(variables={var_name + "_reduced":(('sampling', 'feature_reduced'),X_reduced)})
print(X)

## Apply Model
***

__Create model__

In [ ]:
from sklearn import mixture
k = 7 # number of classes
model = mixture.GaussianMixture(n_components=k, covariance_type='full')
model

__Fit model__

In [ ]:
model.fit(X[var_name + "_reduced"])

__Predict labels__

In [ ]:
X_labels = model.predict(X[var_name + "_reduced"])
X_labels

In [ ]:
X = X.assign(variables={"GMM_labels":(('sampling'),X_labels)})
print(X)
#X

Calculate other __stadistics__

- Predict posterior probability of each component given the data

In [ ]:
X_proba = model.predict_proba(X[var_name + "_reduced"])
np.shape(X_proba)

In [ ]:
X = X.assign(variables={"GMM_post":(('sampling','k'),X_proba)})
print(X)
#X

- Calculate quantiles

In [ ]:
#quantiles we want to calculate
q = [0.05, 0.5, 0.95]

In [ ]:
m_quantiles = X[var_name].where(X['GMM_labels']==0, drop=True).quantile(q, dim='sampling')
for yi in range(1,k):
    m_quantiles = xr.concat((m_quantiles, X[var_name].where(X['GMM_labels']==yi, drop=True).quantile(q, dim='sampling')), dim='k')

In [ ]:
X = X.assign(variables={var_name + "_Q":(('k','quantile','feature'), m_quantiles)})
X = X.assign_coords(coords={'quantile': q})
print(X)

- Calculate quantiles scaled variable

In [ ]:
#quantiles we want to calculate
q = [0.05, 0.5, 0.95]

In [ ]:
m_quantiles = X[var_name].where(X['GMM_labels']==0, drop=True).quantile(q, dim='sampling')
for yi in range(1,k):
    m_quantiles = xr.concat((m_quantiles, X[var_name + '_scaled'].where(X['GMM_labels']==yi, drop=True).quantile(q, dim='sampling')), dim='k')

In [ ]:
X = X.assign(variables={var_name + '_scaled' + "_Q":(('k','quantile','feature'), m_quantiles)})
X = X.assign_coords(coords={'quantile': q})
print(X)

- Calculate robustness

In [ ]:
maxpost = X["GMM_post"].max(dim="k")
K = len(X["GMM_labels"])
robust = (maxpost - 1. / K) * K / (K - 1.)

Plist = [0, 0.33, 0.66, 0.9, .99, 1]
rowl0 = ('Unlikely', 'As likely as not', 'Likely', 'Very Likely', 'Virtually certain')
robust_id = np.digitize(robust, Plist) - 1

In [ ]:
X = X.assign(variables={"GMM_robustness":(('sampling'), robust), "GMM_robustness_cat":(('sampling'), robust_id)})
X["GMM_robustness_cat"].attrs['legend'] = rowl0
print(X)
#X

__Unstack dataset__

In [ ]:
ds_labels = X.unstack('sampling')
#ds_labels = ds_labels.sortby(['lat','lon'])
# same lat and lon values in mask and in results
mask = stacked_mask.unstack()
ds_labels = ds_labels.reindex_like(mask)
print(ds_labels)
#ds_labels

In [ ]:
#copy atributtes
ds_labels.attrs = ds.attrs
ds_labels.lat.attrs = ds.lat.attrs
ds_labels.lon.attrs = ds.lon.attrs
#include time coord for save_BlueCloud function
ds_labels = ds_labels.assign_coords({'time': ds.time.values})
ds_labels.time.attrs = ds.time.attrs
ds_labels

In [ ]:
dims_dict = list(ds_labels.dims.keys())
dims_dict

## Preprocesing plots
***

In [ ]:
P = Plotter_OR(ds_labels, model)

__Scatter plot__

In [ ]:
P.scatter_PDF(var_name = var_name + '_reduced')
P.save_BlueCloud('figures/scatter_PDF_EX.png')

__BIC__

In [ ]:
corr_dist = 70 # correlation distance in km
Nrun = 10 # number of runs for each k
NK = 20 # max number of classes to explore

In [ ]:
BIC, BIC_min = BIC_calculation(X=X, coords_dict={'latitude':'lat', 'longitude':'lon'}, 
                               corr_dist=corr_dist,
                               feature_name='feature_reduced', var_name= var_name + "_reduced",
                               Nrun=Nrun, NK=NK)

In [ ]:
plot_BIC(BIC, NK=NK)
P.save_BlueCloud('figures/BIC_EX.png', bic_fig='yes')

## Plot results
***

In [ ]:
P = Plotter_OR(ds_labels, model)

#### __1) Quantiles time series__

Median and other quantiles representation

In [ ]:
P.tseries_structure(q_variable = var_name + '_Q', start_month=6, ylabel='Temperature (K)')
P.save_BlueCloud('figures/tseries_struc_EX.png')

All median time series in the same plot 

In [ ]:
P.tseries_structure_comp(q_variable = var_name + '_Q', plot_q= 'all', ylabel='Temperature (K)', start_month=6)
P.save_BlueCloud('figures/tseries_struc_comp_EX.png')

Quantiles when temperature is scaled (__how we can interpret this figure?__)

In [ ]:
P.tseries_structure(q_variable = var_name + '_scaled' + '_Q', start_month=6, ylabel='Temperature (K)')
P.save_BlueCloud('figures/tseries_struc_EX_norm.png')

#### __2) Spatial distribution of classes__

In [ ]:
P.spatial_distribution()
P.save_BlueCloud('figures/spatial_distr_EX.png')

#### __3) Robustness__

In [ ]:
P.plot_robustness()
P.save_BlueCloud('figures/robustness_EX.png')

#### __4) Classes pie chart__

In [ ]:
P.pie_classes()
P.save_BlueCloud('figures/pie_chart_EX.png')